In [1]:
# Import dependencies

from splinter import Browser
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [2]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## Scrape book data and load into a DataFrame

In [3]:
# Set the url to be the online bookstore, and open a scraping session
url = 'http://books.toscrape.com/'
browser.visit(url)
browser_url = browser.url

In [4]:
# Initialize empty lists for storing scraped book titles, links, product descriptions, prices
titles = []
full_links = []
product_descriptions = []
prices = []

In [5]:
# Iterate through all pages
for x in range(50):
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Retrieve all elements that contain book information
    articles = soup.find_all('article', class_='product_pod')
    browser_url = '/'.join(browser.url.rstrip('/').split('/')[:-1])

    # Iterate through each book
    for article in articles:
        # Use Beautiful Soup's find() method to navigate and retrieve the anchor tag pertaining to each book
        h3 = article.find('h3')
        link = h3.find('a')

        # Compile the book-specific web address, and handle the landing page's peculiar formatting
        href = link['href']
        if x == 0:
            href = "books.toscrape.com/" + href
        
        # Complete the concatenation of the book page url
        full_link = browser_url + "/" + href
        full_links.append(full_link)

        # Retrieve the title of the book and add it to our list of books
        title = link['title']
        titles.append(title)
        

    # Click the 'Next' button on each page, otherwise print that scraping is complete
    try:
        browser.click_link_by_text('next')
          
    except:
        print("Scraping Complete")

/Users/BootCamp/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:536: FutureWarning: browser.find_link_by_text is deprecated. Use browser.links.find_by_text instead.
  FutureWarning,


Scraping Complete


In [6]:
# Follow each book's link to grab the production description and price for each book
for link in full_links:
    browser.visit(link)
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    
    # Find and append the product description of the current book to our list
    product_description = soup.find_all('p')[3].text
    product_descriptions.append(product_description)
        
    # Find and append the float-formatted price of the current book to our list
    price = float(soup.find_all('p', class_='price_color')[0].text.strip('£'))
    prices.append(price)

In [7]:
# Create a dataframe containins the scraped book data (urls, titles, descriptions, and prices)
books_df = pd.DataFrame(
    {"link": full_links,
     "title": titles,
     "description": product_descriptions,
     "price": prices
     }
)

In [8]:
# Preview the books DataFrame
books_df.head()

,link,title,description,price
0,http://books.toscrape.com/catalogue/a-light-in...,A Light in the Attic,It's hard to imagine a world without A Light i...,51.77
1,http://books.toscrape.com/catalogue/tipping-th...,Tipping the Velvet,"""Erotic and absorbing...Written with starling ...",53.74
2,http://books.toscrape.com/catalogue/soumission...,Soumission,"Dans une France assez proche de la nôtre, un h...",50.10
3,http://books.toscrape.com/catalogue/sharp-obje...,Sharp Objects,"WICKED above her hipbone, GIRL across her hear...",47.82
4,http://books.toscrape.com/catalogue/sapiens-a-...,Sapiens: A Brief History of Humankind,From a renowned historian comes a groundbreaki...,54.23


## Scrape quote data and load into DataFrames

In [9]:
# Set the url to be the quotes website, and open a scraping session
quotes_url = 'http://quotes.toscrape.com/'
browser.visit(quotes_url)

In [10]:
# Initialize an empty list for storing speaker names
speakers = []

In [11]:
# Iterate through each quote
for x in range(1, 11):

    # Parse HTML with Beautiful Soup
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')
    quotes = soup.find_all('small', class_='author')

    # Retrieve the name of the each quote speaker and add it to our list of speakers
    for quote in quotes:
        speakers.append(quote.text)

    # Work through all ten pages of quotes by clicking 'Next' at the end of each page
    if x != 10:
        browser.click_link_by_partial_text('Next')

/Users/BootCamp/anaconda3/lib/python3.7/site-packages/splinter/driver/webdriver/__init__.py:528: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


In [13]:
# Create a dataframe containing the scraped quotes data (speakers)
quotes_df = pd.DataFrame(
    {"quote_speaker": speakers
     }
)

In [14]:
# Create an array that has an ordered array of unique quote speakers
unique_speakers = np.unique(np.array(speakers))

In [15]:
# Convert the unique speakers back into a list which can be loaded into a DataFrame
unique_speakers_list = unique_speakers.tolist()


In [16]:
# Create a dataframe containing the unique speakers
speakers_df = pd.DataFrame(
    {"speaker": unique_speakers_list
     }
)

## Find speakers in book descriptions and build a DataFrame of "matches"

In [49]:
# Initialize empty list for storing titles of the books and names speakers who match
matches_book_title_list = []
matches_speaker_name_list = []

In [50]:
# Loop through the speaker list and record title of each book description mentioning the speaker names

num_speakers = np.arange(0,len(unique_speakers_list))

for i in num_speakers:

    match_book_id_index = books_df[books_df['description'].str.contains(unique_speakers_list[i])].index
    
    if len(match_book_id_index) > 0:       
        
        for foo in match_book_id_index:
            matches_book_title_list.append(books_df.iloc[foo,1])
            matches_speaker_name_list.append(unique_speakers_list[i])
   
    i = i + 1

In [51]:
# Create a DataFrame containing the book titles and speaker names where the product description matches
matches_df = pd.DataFrame({
    "book_title": matches_book_title_list,
    "speaker_name": matches_speaker_name_list
})

## Create database in Postgres using schema.sql

#### This happens offline (i.e., using PgAdmin) prior to running the next code

## Create database connection

In [20]:
# Create a connection to the database we created offline
connection_string = "postgres:yX8nLv1jy7@localhost:5432/etl-project"
engine = create_engine(f'postgresql://{connection_string}')

In [21]:
# Confirm tables
engine.table_names()

['quotes', 'books', 'matches', 'speakers']

## Load DataFrames into database

In [22]:
# Load books into Database
books_df.to_sql(name='books', con=engine, if_exists='append', index=False) 

In [23]:
# Load quotes into Database
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False) 

In [24]:
# Load speakers into Database
speakers_df.to_sql(name='speakers', con=engine, if_exists='append', index=False) 

In [42]:
# Load books table back into DataFrames so we can grab database IDs of records of interest
books_2_df = pd.read_sql("select * from books", con=engine)

In [44]:
# Load speakers table back into DataFrames so we can grab database IDs of records of interest
speakers_2_df = pd.read_sql("select * from speakers", con=engine)

In [58]:
# Merge matches DataFrame with books from DataBase
matches_df_2 = matches_df.merge(books_2_df, left_on="book_title", right_on="title")

In [60]:
# Merge evolved matches DataFrame with speakers from DataBase
matches_df_3 = matches_df_2.merge(speakers_2_df, left_on="speaker_name", right_on="speaker")

In [64]:
# Create a filtered matches dataframe from specific columns
matches_cols = ["id_x", "id_y"]
matches_transformed_df = matches_df_3[matches_cols].copy()

# Rename the column headers
matches_transformed_df = matches_transformed_df.rename(columns={"id_x": "book_id", "id_y": "speaker_id"})

In [66]:
# Load matches into Database
matches_transformed_df.to_sql(name='matches', con=engine, if_exists='append', index=False) 